In [1]:
import xarray as xr
import pandas as pd
import numpy as np

In [2]:
from utils import *

ModuleNotFoundError: No module named 'utils'

In [ ]:

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# ----     load data stored in casper     ----
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#-------Gridcell Landareas Data-----
# reading, storing, subsetting
landarea_file = '/glade/campaign/cgd/tss/projects/PPE/helpers/sparsegrid_landarea.nc'

landarea_ds = xr.open_dataset(landarea_file)

landarea = landarea_ds['landarea']

#-------Dummy Variable Data---------
# dummy data to have stored for preloaded visual on 
dummy_filepath = '/glade/campaign/cgd/tss/projects/PPE/PPEn11_OAAT/CTL2010/hist/PPEn11_CTL2010_OAAT0000.clm2.h0.2005-02-01-00000.nc'

# dummy data described by filepath stored as ds
ds = xr.open_dataset(dummy_filepath)

# dummy variable data subset as data array. using LNC for linearity
da = ds['LNC']

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# ----     correct time-parsing bug       ----
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
def fix_time(da):
    '''fix CESM monthly time-parsing bug'''
    yr0 = str(da['time.year'][0].values)
    da['time'] = xr.cftime_range(yr0,periods=len(da.time),freq='MS',calendar='noleap')
    return da


#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# ----  weigh dummy landarea by gridcell  ----
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#------Weight Gridcells by Landarea---
def weight_landarea_gridcells(da,landarea):

    # weigh landarea variable by mean of gridcell dimension
    weighted_avg_area = da.weighted(landarea).mean(dim = 'gridcell')

    return da   # QUESTION: Should we return ds so that we can call this later thru utils? (changed from weighted_avg_area)


#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# ----     weigh dummy data time dim      ----
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#------Weighted Averages by Time---
def yearly_weighted_average(da):
    # Get the array of number of days from the main dataset
    days_in_month = da['time.daysinmonth']

    # Multiply each month's data by corresponding days in month
    weighted_sum = (days_in_month*da).groupby("time.year").sum(dim = 'time')

    # Total days in the year
    total_days = days_in_month.groupby("time.year").sum(dim = 'time')

    # Calculate weighted average for the year
    da['time'] = weighted_sum / total_days            # QUESTION: Is this right?

    return da

### Function to Automate Global Avg Weighted Landarea by Gridcell


In [ ]:
filepath_example = '/glade/campaign/cgd/tss/projects/PPE/PPEn11_OAAT/CTL2010/hist/PPEn11_CTL2010_OAAT0000.clm2.h0.2005-02-01-00000.nc'

#-------Gridcell Landareas Data-----

# reading, storing, subsetting
file2 = '/glade/campaign/cgd/tss/projects/PPE/helpers/sparsegrid_landarea.nc'
ds2 = xr.open_dataset(file2)
landarea = ds2['landarea']

#-------Variable Data-----------------
# variable data described by filepath stored as ds
ds = xr.open_dataset(filepath_example)
da = ds[variable]

#------Weight Gridcells by Landarea---
def weight_landarea_gridcells(da,landarea):
    weighted_avg_area = da.weighted(landarea).mean(dim = 'gridcell')
    return weighted_avg_area

In [ ]:
weight_landarea_gridcells(da,landarea)

### Function to Automate Weighted Time

In [ ]:
def yearly_weighted_average(da):
    # Get the array of number of days from the main dataset
    days_in_month = da['time.daysinmonth']
    weighted_sum = (days_in_month*da).groupby("time.year").sum(dim = 'time') # Multiply each month's data by corresponding days in month
    total_days = days_in_month.groupby("time.year").sum(dim = 'time')  # Total days in the year
    return weighted_sum / total_days  # Calculate weighted average for the year

In [ ]:
yearly_weighted_average(da)

In [ ]:
filepath_example = '/glade/campaign/cgd/tss/projects/PPE/PPEn11_OAAT/CTL2010/hist/PPEn11_CTL2010_OAAT0000.clm2.h0.2005-02-01-00000.nc'
ds = xr.open_dataset(filepath_example)

In [ ]:
ds.TSA.dims

In [ ]:
file2 = '/glade/campaign/cgd/tss/projects/PPE/helpers/sparsegrid_landarea.nc'
ds2 = xr.open_dataset(file2)
landarea = ds2['landarea']
weighted_avg_area = ds.TSA.weighted(landarea).mean(dim = 'gridcell')
weighted_avg_area

In [ ]:
(1/landarea.sum())*(landarea*ds.TSA).sum(dim = 'gridcell')

In [ ]:
# dummy variable filepath for TSA,
filepath_example = '/glade/campaign/cgd/tss/projects/PPE/PPEn11_OAAT/CTL2010/hist/PPEn11_CTL2010_OAAT0000.clm2.h0.2005-02-01-00000.nc'

# testing our function works!
ds = weight_landarea_gridcells(filepath_example)
ds

In [ ]:

# subsetting dummy variable
tsa = ds['TSA']

# calculate global mean over time
tsa_avg = tsa.mean(dim = 'time')

# plot mean tsa overtime of weighted gridcells
plt.figure(figsize = (9,4))
plt.scatter(ds.grid1d_lon,
            ds.grid1d_lat,
            c = tsa_avg,
           cmap = 'RdPu',
           alpha = 0.5)
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Default Simulation: OAAT0000')
plt.colorbar(label = 'Average TSA: 2m air temperature (K)')

In [ ]:
ds.info()

##### Visualization

##### Avg TSA 

Want to take the mean of variable `TSA` so we can use this as a variable to color map with later.

In [ ]:
tsa = ds['TSA']

tsa

In [ ]:
tsa_avg = tsa.mean(dim = 'time')

In [ ]:
plt.figure(figsize = (9,4))
plt.scatter(ds.grid1d_lon,
            ds.grid1d_lat,
            c = tsa_avg,
           cmap = 'RdPu',
           alpha = 0.5)
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Default Simulation: OAAT0000')
plt.colorbar(label = 'Average TSA: 2m air temperature (K)')

In [ ]:
# subsetting dummy variable
tsa = ds['TSA']

# calculate global mean over time
tsa_avg = tsa.mean(dim = 'time')

# plot mean tsa overtime of weighted gridcells
plt.figure(figsize = (9,4))
plt.scatter(ds.grid1d_lon,
            ds.grid1d_lat,
            c = tsa_avg,
           cmap = 'RdPu',
           alpha = 0.5)
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Default Simulation: OAAT0000')
plt.colorbar(label = 'Average TSA: 2m air temperature (K)')

## Reading cluster data

In [ ]:
# read all simulations as a list
cluster_list = sorted(glob.glob('/glade/campaign/cgd/tss/projects/PPE/PPEn11_LHC/transient/hist/PPEn11_transient_LHC[0][0-5][0-9][0-9].clm2.h0.2005-02-01-00000.nc'))
cluster_list = cluster_list[1:len(cluster_list)]
#cluster_list

In [ ]:
# creating cluster
#read the list and load it for the notebook
LNC = xr.open_mfdataset(cluster_list, 
                                   combine='nested',
                                   parallel=True, 
                                   concat_dim="ens")

In [ ]:
# confirming that the ensemble dataset loaded correctly! yay! ens: 500 
LNC

## Linear Regression

#### Combining da into ds for linear reg
This produces three dimensions: variable, lat, long. lin. reg only works for 2D or less.

In [ ]:
# Combining into a Dataset
test = xr.Dataset({"LEAFCN": LEAFCN_da, "LNC": LNC_da})

In [ ]:
# Assuming leafcn_da and LNC_da are your pre-processed data arrays
# Make sure that leafcn_da and LNC_da have the same length and are 1D arrays
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# ----      Split Data 90/10        ----
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
x_train, X_test, y_train, y_test = train_test_split(test['LEAFCN'], test['LNC'], test_size=0.1, random_state=0)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# ----      Linear Regression       ----
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
regressor = LinearRegression()
regressor.fit(x_train, y_train)

#### Dropping NA

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# ----     Subsetting Dummy Data    ----
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# only the da was pre_processed, so we're weighing the ds here and making an array
leafcn_da = ds['LEAFCN'].dropna('gridcell')

leafcn_da

# only the da was pre_processed, so we're weighing the ds here and making an array
leafcn_da = yearly_weighted_average(leafcn_da)
#.fix_time()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
#from sklearn_xarray import Stacker, Select


#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# ----      Split Data 90/10        ----
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
da = da.dropna('gridcell')

X_train, X_test, y_train, y_test = train_test_split(leafcn_da, da, test_size = 0.1, random_state=0)


#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# ----      Linear Regression       ----
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
regressor = LinearRegression()
regressor.fit(X_train, y_train)

LinearRegression()

y_pred = regressor.predict(X_test)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# ----    Plot Linear Regression    ----
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
plt.scatter(X_train, y_train,color='g')
plt.plot(X_test, y_pred,color='k')

plt.show()
plot.show() 

In [ ]:
# calculate global mean over time
LNC_avg = da.mean(dim = 'time')

# plot mean tsa overtime of weighted gridcells
plt.figure(figsize = (9,4))
plt.scatter(leafcn_da,
            LNC_avg,
            c = LNC_avg,
           cmap = 'RdPu',
           alpha = 0.5)
plt.xlabel('Leaf Carbon Nitrogen Ratio')
plt.ylabel('Leaf Nitrogen Concentration')
plt.title('Default Simulation: OAAT0000')
plt.colorbar(label = 'Concenctration (units to be added)')


## Imputer NaN filler for Linear Regression
Linear Regression only works when both of the data arrays being passed do not contain NaN values. This might also be the issue preventing us from plotting above. I am going to try using the `SimpleImputer()` function to replace missing values with the mean values of this column. A variety of stats options may be selected to fill this instead. I've choosen mean for now. This still didn't work

In [ ]:
# Assuming leafcn_da and LNC_da are your pre-processed data arrays
# Make sure that leafcn_da and LNC_da have the same length and are 1D arrays
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# ----      Split Data 90/10        ----
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
X_train, X_test, y_train, y_test = train_test_split(leafcn_da, LNC_da, test_size=0.1, random_state=0)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# ----         Removing NaN         ----
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Handle NaN values using SimpleImputer
imputer = SimpleImputer(strategy='mean')  # You can choose other strategies like 'median' or 'most_frequent'
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputer = imputer.fit_transform(X_test)

y_train_imputed = imputer.fit_transform(y_train)
y_test_imputed = imputer.fit_transform(y_test)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# ----      Linear Regression       ----
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
regressor = LinearRegression()
regressor.fit(X_train_imputed, y_train_imputed)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# ----       Make Predictions       ----
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
y_pred = regressor.predict(X_test_imputed)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# ----    Plot Linear Regression    ----
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
plt.scatter(X_train_imputed, y_train_imputed, color='g', label='Training Data')
plt.scatter(X_test_imputed, y_test_imputed, color='b', label='Testing Data')
plt.plot(X_test_imputed, y_pred, color='k', linewidth=2, label='Linear Regression')

plt.xlabel('leafcn')
plt.ylabel('LNC_da')
plt.title('Linear Regression for LNC')
plt.legend()
plt.show()